In [ ]:
import numpy as np
import plotly.express as px
def plot_embedding(embedding, labels=None):
    """
    Plot a 2D embedding with optional labels for coloring.

    Args:
        embedding (ndarray): shape (n_samples, 2)
        labels (array-like, optional): labels for coloring points
    """
    if labels is None:
        labels = np.zeros(embedding.shape[0])  # default: all same


    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color=labels.astype(str),  # convert to string for discrete legend
        title="2D Embedding Visualization",
        labels={"x": "Dim 1", "y": "Dim 2", "color": "Label"}
    )


    fig.update_layout(
        width=1000,
        height=800,
        title="Custom size Plotly figure"
    )


    fig.show()



In [ ]:
import data_call
import numpy as np
d_name='Muraro'
X,label=data_call.get_dataset(d_name, mode='PCA')

In [ ]:
import scanpy as sc
nk_path='/Users/chandrasekharmukherjee/PycharmProjects/Personal/cplearn_biotest/NK_cells.h5ad'
adata = sc.read_h5ad(nk_path)

print("done")

X=adata.obsm['X_HARMONY']
label=adata.obs['ident'].to_numpy()
from scipy.sparse import issparse
X_umap=adata.obsm['X_UMAP']

X_cg = adata.X.toarray() if issparse(adata.X) else adata.X
gene_names = adata.var_names.values

print(X_cg.shape,gene_names.shape,gene_names[0:5])

In [ ]:
f_path='/Users/chandrasekharmukherjee/Data/scRNA_IBD/'
d_path=['bcells_final.h5ad',
'cd4tcells_final.h5ad',
'cd8tcells_final.h5ad',
'epicolonic_final.h5ad',
'fibperi_final.h5ad',
'ilc_final.h5ad',
'ilealepi_final.h5ad',
'myeloid_final.h5ad',
'plasmacells_final.h5ad']

subset_number=1

import scanpy as sc
adata=sc.read_h5ad(f_path+d_path[subset_number])
X=adata.obsm['X_harmony']
n=X.shape[0]
print(n)
label=adata.obs['final_analysis'].to_numpy()
pt_label=adata.obs['Patient'].to_numpy()

In [ ]:
from cplearn_v4.corespect import CorespectModel
from cplearn_v4.corespect.config import (
    FlowRankConfig,
    StableCoreConfig,
FineGrainedConfig)

# ,
#     FineGrainedConfig,
#     PropagationConfig
# )

In [ ]:
flow_cfg = FlowRankConfig(q=20, r=10)
stable_cfg = StableCoreConfig(core_frac=0.2, resolution=1.5, ng_num=10, densification='rw')
fine_cfg= FineGrainedConfig(fine_grain_densification='rw')

model = CorespectModel(
    X,
    flowrank_cfg=flow_cfg,
    stable_cfg=stable_cfg,
    fine_cfg=fine_cfg
)

print("Running CoreSPECT\n")

corespect_obj=model.run(fine_grained=True)

In [ ]:
import umap
reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

In [ ]:
plot_embedding(X_umap,label)

In [ ]:
print([len(layer) for layer in corespect_obj.layers_])

In [ ]:
print(len(corespect_obj.layers_[0]))
plot_embedding(X_umap[corespect_obj.layers_[0]],label[corespect_obj.layers_[0]])
plot_embedding(X_umap[corespect_obj.layers_[1]],label[corespect_obj.layers_[1]])

In [ ]:
print(len(corespect_obj.layers_[0]))
plot_embedding(X_umap[corespect_obj.layers_[0]],label[corespect_obj.layers_[0]])

In [ ]:
import umap
reducer=umap.UMAP()
X_umap_core=reducer.fit_transform(X[corespect_obj.layers_[0]])
plot_embedding(X_umap_core,label[corespect_obj.layers_[0]])